In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langchain-openai

In [ ]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# FILEPATH: /Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly_live_training_agents/notebooks/1.3-intro-agents-with-langchain.ipynb
# SET YOUR API KEYS FOR LANGSMITH AND TAVILY TO RUN THIS NOTEBOOK!
# langsmith: https://docs.smith.langchain.com/#:~:text=install%20-U%20langsmith-,Create%20an%20API%20key,click%20Create%20API%20Key.,-Setup%20your%20environment
# tavily: https://app.tavily.com/sign-in
# openai: https://platform.openai.com/docs/quickstart
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "your-api-key"
os.environ["TAVILY_API_KEY"] = "your-api-key"
os.environ["OPENAI_API_KEY"] = 'your-api-key'

Personal Assistant Agent
- Accees to emails
- acceess to my terminal for controlling different things like creating files etc...
- Have the assistant create our tasks for us

In [1]:
from langchain.agents import create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.tools import tool
from langchain.agents import AgentExecutor
from langchain_community.document_loaders import PyPDFLoader
import subprocess
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import subprocess
import shlex

In [2]:
@tool
def send_email(destination_email, message_content, subject_content):
    "This function takes in destination email and the contents and sends out an email."
    # Sender email address
    sender_email = "lucasenkrateia@gmail.com"

    # Email Subject
    subject = subject_content

    # Email body
    body = message_content

    # SMTP server credentials
    smtp_server = "smtp.gmail.com"
    smtp_port = 587  # For starttls
    smtp_user = sender_email
    smtp_password = os.environ["GMAIL_PWD"]

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = destination_email
    message["Subject"] = subject

    # Add body to email
    message.attach(MIMEText(body, "plain"))

    # Convert message to string for sending
    text = message.as_string()

    # Send email
    try:
        # Connect to the server
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Secure the connection
        server.login(smtp_user, smtp_password)
        
        # Send the email
        server.sendmail(sender_email, destination_email, text)
        print("Email sent successfully")
    except Exception as e:
        # Print any error messages to stdout
        print(f"Error: {e}")
    finally:
        server.quit()


@tool
def execute_bash_command(command):
    """Executes a bash command."""
    result = subprocess.run(command, shell=True)
    output = result.stdout
    return output

@tool
def read_txt_tool(file_path):
    """Reads contents of a .txt file."""
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

@tool
def summarize_pdf(pdfpath):
    """Summarizes the contents of a pdf"""
    loader = PyPDFLoader(pdfpath)
    documents = loader.load()
    contents = ""
    for doc in documents:
        contents += doc.page_content + "\n"
    
    summary = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").invoke(f"Summarize this content into simplified bullet points: {contents}")
    
    return summary.content
    
tools = [execute_bash_command, send_email, read_txt_tool, summarize_pdf]

In [3]:
prompt = hub.pull("hwchase17/openai-functions-agent")

prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

In [4]:
llm = ChatOpenAI(model="gpt-4-0125-preview")

In [5]:
agent = create_openai_tools_agent(llm, tools, prompt)

In [6]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke({"input": "1. Create a file named tasks-for-lucas.txt containing 10 automation tasks you find interesting. 2. Send email to '''lucasbnsoares@hotmail.com''' with the top 3 tasks from that file.",
                             "agent_scratchpad": []})



> Entering new AgentExecutor chain...

Invoking: `execute_bash_command` with `{'command': 'echo "1. Automate the summarization of PDF documents for quick review.\n2. Automate the process of converting speech to text for meeting notes.\n3. Create a bot that automatically responds to specific emails based on predefined criteria.\n4. Develop a system that automatically backs up important files to the cloud at scheduled intervals.\n5. Implement a tool that tracks and reports on changes to specific websites or web pages.\n6. Create a script that automatically organizes files in a folder based on their type or content.\n7. Develop a program that can automatically edit videos based on user-defined parameters.\n8. Automate the process of data entry from scanned documents into a database.\n9. Create a bot that can automatically post content to social media platforms based on a schedule.\n10. Implement an AI-driven assistant that can help manage and prioritize emails." > tasks-for-lucas.txt'}`

{'input': "1. Create a file named tasks-for-lucas.txt containing 10 automation tasks you find interesting. 2. Send email to '''lucasbnsoares@hotmail.com''' with the top 3 tasks from that file.",
 'agent_scratchpad': [],
 'output': "I've successfully created a file named `tasks-for-lucas.txt` containing 10 interesting automation tasks and sent an email to `lucasbnsoares@hotmail.com` with the top 3 tasks from that file."}

In [8]:
!cat tasks-for-lucas.txt

1. Automate the summarization of PDF documents for quick review.
2. Automate the process of converting speech to text for meeting notes.
3. Create a bot that automatically responds to specific emails based on predefined criteria.
4. Develop a system that automatically backs up important files to the cloud at scheduled intervals.
5. Implement a tool that tracks and reports on changes to specific websites or web pages.
6. Create a script that automatically organizes files in a folder based on their type or content.
7. Develop a program that can automatically edit videos based on user-defined parameters.
8. Automate the process of data entry from scanned documents into a database.
9. Create a bot that can automatically post content to social media platforms based on a schedule.
10. Implement an AI-driven assistant that can help manage and prioritize emails.


![](2024-03-14-19-26-00.png)

In [ ]:
agent_executor.invoke({"input": "Create a folder named `lucas-test`. Inside that folder create a file named lucas-test.txt containing a random quote from a famous philosopher. Send an email to '''lucasbnsoares@hotmail.com''' with that quote.",
                             "agent_scratchpad": []})

agent_executor.invoke({"input": """1. Read the contents of this pdf: research report located in /Users/greatmaster/Desktop/projects/research_reports/prompt_engineering_techniques.pdf and save it to ./paper-output.txt. 2. Read the contents of the .txt file. 3. Send the output '''lucasbnsoares@hotmail.com'''""",
                                "agent_scratchpad": []})

In [ ]:
# Let's add a summary to this pipeline.

# Starts making mistakes when I add just summarization to the prompt, and the error is not even at that level.
# So just saying summarize makes the model produce errors in other parts of the pipeline that were working fine previously.
agent_executor.invoke({"input": """1. Summarize this pdf: /Users/greatmaster/Desktop/projects/research_reports/prompt_engineering_techniques.pdf contents to a concise instructive paragraph. 2. Send the output to '''lucasbnsoares@hotmail.com'''""",
                                "agent_scratchpad": []})